In [1]:
# # # For quickly testing when working on external libraries
# %load_ext autoreload
# %autoreload 2

# TopoCluster Regression

This is a slightly different take on regression than what we've commonly being doing. Instead of making a network that will output the predicted topo-cluster energy, we will make one that will output the predicted *ratio* between true and reco energies. In other words, this network will predict the "calibration factor" for our topo-cluster energy.

What's the difference? It might sound like a trivial change, but the ratio is a unitless quantity. So this might help with the problem of having to cover many orders of magnitude in outputs. Now, our "classic" regression networks were dealing with this by performing some scaling of inputs:

$$
\begin{align}
E_\text{reco}  &\rightarrow \ln{ \left (E_\text{reco} \right)} \;, \\
E_\text{truth}  &\rightarrow \ln{ \left (E_\text{truth} \right)} \;\; \text{(target)},
\end{align}
$$

The output of the network, $x$, is then converted to the predicted energy $E_\text{pred}$ via the inverse transformation:

$$
x \rightarrow e^{x} = E_\text{pred}
$$

Thus if there's some small error in $x$, $x \rightarrow x + \delta x$, this may result in a large error in $E_\text{pred}$:

$$
e^{x + \delta x} = e^{\delta x} E_\text{pred} \; .
$$

For example, if we're trying to get an energy of $100 \, \text{[GeV]}$ for a particular cluster, but our network ouput ($x$) is $10\%$ too large, this will lead to a $\sim 50\%$ error in $E_\text{pred}$.

#### 1) Initial setup

In [2]:
# ML fitting/loading/saving settings
overwriteModel = False # If true, force training. If false, load the specified model if it already exists.

finishTraining = False

Now let's choose our training data.

In [3]:
subdir = 'pion_full' # name for subdir holding models/plots
h5_name_suffix = 'tdata_full' # name for HDF5 files containing selected training/validation/testing data.

For convenience, we can define some training settings here for our different models. Note that we'll need to expand this if we add new models to the list.

Also note that some model-specific settings are only adjusted further down in the code, where each model is defined.

In [4]:
training_settings = {
    'baseline_dnn':  {'lr':1.0e-5, 'epochs':200, 'patience':40,  'batch_size':200, 'skip':False},
    'simple_dnn':    {'lr':5.0e-5, 'epochs':200, 'patience':40,  'batch_size':200, 'skip':False},
    'simple_cnn':    {'lr':1.0e-4, 'epochs':200, 'patience':40,  'batch_size':200, 'skip':False},
    'split_emb_cnn': {'lr':1.0e-4, 'epochs':200, 'patience':40,  'batch_size':200, 'skip':False},
    'resnet':        {'lr':1.0e-5, 'epochs':200, 'patience':40,  'batch_size':200, 'skip':True },
    'lorentz':       {'lr':1.0e-5, 'epochs':400, 'patience':300, 'batch_size':200, 'skip':True }
}

In [5]:
# Import some basic libraries.
import sys, os, glob, pathlib
import numpy as np
import matplotlib.pyplot as plt
import uproot as ur
#import ROOT as rt # optionally used for plotting
#import joblib as jl # for saving scalers

# Import our resolution utilities
path_prefix = os.getcwd() + '/../'
if(path_prefix not in sys.path): sys.path.append(path_prefix)
from util import resolution_util as ru
from util import plot_util       as pu
from util import ml_util         as mu
from util import qol_util        as qu
from util import io_util         as iu

# Custom tensorflow.keras callbacks
from util.keras.callbacks import GetCallbacks

# Regression-specific utils
from util.regression import data_util as rdu
from util.regression import plot_util as rpu
from util.regression import training_util as rtu

#rt.gStyle.SetOptStat(0)
# use our custom dark style for plots
plotstyle = qu.PlotStyle('dark')
plotstyle.SetStyle() # still need to manually adjust legends, paves

plotpath = path_prefix + 'regression2/Plots/{}/'.format(subdir)
modelpath = path_prefix + 'regression2/Models/{}/'.format(subdir)
paths = [plotpath, modelpath]
for path in [plotpath, modelpath]:
    try: os.makedirs(path)
    except: pass

# metadata
layers = ["EMB1", "EMB2", "EMB3", "TileBar0", "TileBar1", "TileBar2"]
cell_size_phi = [0.098, 0.0245, 0.0245, 0.1, 0.1, 0.1]
cell_size_eta = [0.0031, 0.025, 0.05, 0.1, 0.1, 0.2]
len_phi = [4, 16, 16, 4, 4, 4]
len_eta = [128, 16, 8, 4, 4, 2]
cell_shapes = {layers[i]:(len_eta[i],len_phi[i]) for i in range(len(layers))}

Welcome to JupyROOT 6.24/02


2021-09-30 13:14:27.642974: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [6]:
# fancy display names for each pion type
pi_latex = {
    'p0': '\(\pi^{0}\)',
    'pp': '\(\pi^{\pm}\)',
}
pi_text = {
    'p0': 'pi0',
    'pp': 'pi+/-'
}

# Plotting settings
# xkcd -- turn this on for fun-looking (but marginally less useful) plots
use_xkcd = False
if(use_xkcd):
    mode = 'light'
    plt.xkcd(scale=.75,length=100,randomness=1)
    
# plotting style -- manages our color palette and object colors
mode = 'dark' # for publications, use "light"
plotstyle = qu.PlotStyle(mode)
    
# some matplotlib-specific stuff
params = {'legend.fontsize': 13,
          'axes.labelsize': 18}
plt.rcParams.update(params)

#### 2) Get the data.

In [7]:
inputpath=path_prefix+'data/pion/'
rootfiles = {        
    'p0':inputpath + 'user.mswiatlo.900246.PG_singlepi0_logE0p2to2000.recon.ESD.e8312_e7400_s3170_r12383.images_v01.1_OutputStream/*.root',
    'pp':inputpath + 'user.mswiatlo.900247.PG_singlepion_logE0p2to2000.recon.ESD.e8312_e7400_s3170_r12383.images_v01.1_OutputStream/*.root'
}

branches = ['clusterE', 'clusterPt', 'clusterEta', 'cluster_ENG_CALIB_TOT', 'cluster_ENG_CALIB_DEAD_TOT', 'cluster_ENG_CALIB_OUT_T']

In [8]:
cuts = [
    ['cluster_ENG_CALIB_TOT',.2,'lower'],
    ['clusterEta',(-0.7,0.7),'window']
]

cut_distributions = [x[0] for x in cuts]
cut_values = [x[1] for x in cuts]
cut_types = [x[2] for x in cuts]

n_max = -1 # we usually use 300k, or 600k. 50k for 60GeV_mini dataset. Setting -1 means use max possible data.

In [9]:
# Prepare data
h5_name = inputpath + h5_name_suffix

pdata,pcells = mu.setupPionData(
    rootfiles, 
    branches=branches, 
    layers=layers, 
    balance_data=True, 
    n_max=n_max,
    verbose=True,
    load=True,
    save=True,
    filename=h5_name,
    match_distribution='cluster_ENG_CALIB_TOT',
    match_binning=(20000,0.,2000.),
    cut_distributions=cut_distributions,
    cut_values=cut_values,
    cut_types=cut_types
)

total = np.sum([len(x) for x in pdata.values()],dtype=int)
for key,frame in pdata.items():
    n = len(frame)
    print("Number of {a:<7} events: {b:>10}\t({c:.1f}%)".format(a=pi_text[key], b = n, c = 100. * n / total))
print("Total: {}".format(total))

# Create/get training/validation/testing indices.
pdata = rdu.DataPrep(pdata,
                     trainfrac=0.7,
                     filename=h5_name
                    )

Loading pandas DataFrame and calo images from /local/home/jano/ml4pions/LCStudies/regression2/../data/pion/tdata_full_frame.h5 and /local/home/jano/ml4pions/LCStudies/regression2/../data/pion/tdata_full_images.h5.
Number of pi0     events:    2329624	(50.0%)
Number of pi+/-   events:    2329624	(50.0%)
Total: 4659248
Loading indices for key p0 from /local/home/jano/ml4pions/LCStudies/regression2/../data/pion/tdata_full_indices.h5.
Loading indices for key pp from /local/home/jano/ml4pions/LCStudies/regression2/../data/pion/tdata_full_indices.h5.


### Defining regression variables

Beyond the information already present in the data, we may want to pre-compute some extra regression inputs.

The difference between computing them here, on the whole dataset, versus doing it in batch as part of the network itself, is that we can also define some `scalers` based on these variables. This will allow us to scale them across the dataset, for example to get them into the interval of \[0,1\].

In [ ]:
m = 1.
b = 1.
EnergyMapping = iu.LinLogMapping(b=b,m=m)

It looks like our `LinLogMapping` is pretty numerically stable as long as $b \gg m$. I think this is the expected behaviour, given the instabilities with logarithms that we've seen before.

In [ ]:
# Some regression vars.
# Note that the mapping functions can probably be sped up with numba, but might throw errors during plotting above. Need to look into this.
for key,frame in pdata.items():
    frame['logE'] = EnergyMapping.Forward(frame['clusterE'].to_numpy()) # log of reco energy, possible network input
    frame['clusterEtaAbs'] = np.abs(frame['clusterEta'].to_numpy()) # absolute value of eta, possible network input
    #frame['logECalib'] = EnergyMapping.Forward(frame['cluster_ENG_CALIB_TOT'].to_numpy()) # No longer our regression target
    
    frame['cluster_E_true'] = frame['cluster_ENG_CALIB_TOT'].to_numpy() + frame['cluster_ENG_CALIB_DEAD_TOT'].to_numpy() + frame['cluster_ENG_CALIB_OUT_T'].to_numpy()

    # Now get the ratio of "truth" to reco energy. Be mindful of zeros in denominator!
    x = frame['clusterE'].to_numpy()
    x[x==0] = 1.
        
    frame['ratioE'] = frame['cluster_E_true'].to_numpy() / x

In [ ]:
scaler_file = modelpath + 'scalers.save'
scaler_branches = ['logE', 'clusterEtaAbs']
scalers = mu.setupScalers(pdata, scaler_branches, scaler_file)

Let's make plots of any vars that are rescaled using our StandardScalers, to see what the rescaling has done. (We won't put axis labels since the units are kind of funny anyway...)

In [ ]:
# Let's make a histogram of the regression vars, before and after scaling.
rvars = ['logE', 's_logE']
ranges = {
    'logE':(120,-2.,10.),
    's_logE':(100,-2.,2.)
}
fig, ax = plt.subplots(1,len(rvars),figsize=(7.5 * len(rvars),5))
if(type(ax) != np.ndarray): ax = [ax]

for i,rvar in enumerate(rvars):
    vals = {}
    for key,frame in pdata.items():
        vals[key] = frame[rvar].to_numpy()
    pu.histogramOverlay(ax[i], vals.values(), list(vals.keys()), rvar, 'Fractional count',
                        x_min = ranges[rvar][1], x_max = ranges[rvar][2], xbins = ranges[rvar][0],
                        normed = True, y_log = True,
                        ps = plotstyle
                       )
plt.show()

Let's also make a plot of our regression target, to see what the distribution looks like.

In [ ]:
# Let's make a histogram of the regression vars, before and after scaling.
rvars = ['ratioE']
ranges = {
    'ratioE':(200,0.,20.),
}

fig, ax = plt.subplots(1,len(rvars),figsize=(7.5 * len(rvars),5))
if(type(ax) != np.ndarray): ax = [ax]

for i,rvar in enumerate(rvars):
    vals = {}
    for key,frame in pdata.items():
        vals[key] = frame[rvar].to_numpy()
    pu.histogramOverlay(ax[i], vals.values(), list(vals.keys()), rvar, 'Fractional count',
                        x_min = ranges[rvar][1], x_max = ranges[rvar][2], xbins = ranges[rvar][0],
                        normed = True, y_log = True,
                        ps = plotstyle
                       )
plt.show()

As we might have suspected, there is more spread for $\pi^\pm$ than for $\pi^0$. In other words, the charged pion response is worse, and so it will make for a harder regression target. This is consistent with what we've seen elsewhere.

We may optionally perform some re-weighting of our training events. If using the `pion_reweighted` strategy, we will re-weight our single-pion training data to match the topo-cluster $p_T$ spectrum of our jet data.

In [ ]:
# TODO: consider re-adding some sample weighting schemes
sample_weights = {
    key: np.full(np.sum(frame['train'].to_numpy()),1.)
    for key,frame in pdata.items()
}

## Tensorflow/Keras Prep

In this workflow we have the ability to train a number of models -- some will require additional data setup. Here, we have some basic setup they will all use.

In [ ]:
models = {} # keep track of the models -- note that we train multiple instances of each model
histories = {} # keep track of histories of the models we train
regressors = {} # keep track of the trained models (will be KerasRegressor objects)
energy_ratio_names = {} # keep track of the names of regressed variables

model_filename_suffixes = {
    'pp':'_charged',
    'p0':'_neutral'
}

energy_name_prefix = 'ratio_pred_'

#from keras.wrappers.scikit_learn import KerasRegressor # scikit_learn wrapper -- why do we use this, vs. native tf.keras approach like in classification notebook?
#from tensorflow.keras.models import load_model

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # disable some of the tensorflow info printouts, only display errors
import tensorflow as tf
#strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0"])
#ngpu = strategy.num_replicas_in_sync
#print ('Number of devices: {}'.format(ngpu))
ngpu = 1

#from util.regression.models import baseline_nn_All_model, simple_dnn, resnet, resnet_wide
from util.regression.models import baseline_nn_model, depth_network, simple_cnn, split_emb_cnn, resnet, lorentz_net

## Training "baseline_dnn" model

Here we train a simple, fully-connected neural network that uses the calorimeter cells as input, along with reco energy and $\eta$.

In [ ]:
# Baseline model
model_key = 'baseline_dnn'
skip = training_settings[model_key]['skip']
if(not skip):
    print('Starting baseline model.')
    
    # Data preparation
    All_input = rdu.CombinedInput(pdata,
                                  pcells,
                                  branches = ['s_logE','s_clusterEtaAbs']
                                 )
    # ----------------------------------------------------------------

    energy_ratio_names[model_key] = energy_name_prefix + model_key
    lr = training_settings[model_key]['lr']
    decay = 0. # lr decay *within* each epoch
    dropout = -1. # < 0 -> no dropout
    models[model_key] = baseline_nn_model(lr=lr, decay=decay, dropout=dropout)

    # Set our training hyper-parameters.
    batch_size = training_settings[model_key]['batch_size']
    epochs = training_settings[model_key]['epochs']
    gamma = .1 # lr decay between epochs (via scheduler)
    patience = training_settings[model_key]['patience']
    min_delta = 0.0005
    verbose = 1
    regressors[model_key] = {}
    histories[model_key] = {}

    # Load/train the models, and evaluate them on all the data.
    for key in All_input.keys():
        print('{}: {}'.format(model_key, pi_text[key]))

        tidx = pdata[key]['train']
        vidx = pdata[key]['val'] 

        model_dir = ''.join([modelpath, model_key])
        model_filename = '{}/{}{}.h5'.format(model_dir,model_key,model_filename_suffixes[key])

        regressors[model_key][key], histories[model_key][key] = rtu.TrainNetwork(
            model=models[model_key],
            modelfile = model_filename,
            x_train = All_input[key][tidx],
            y_train = pdata[key]['ratioE'][tidx],
            x_valid = All_input[key][vidx],
            y_valid = pdata[key]['ratioE'][vidx],
            sample_weight=sample_weights[key],
            callbacks = GetCallbacks(model_filename, append=True, use_decay=True, gamma=gamma, min_delta=min_delta, patience=patience), 
            epochs=epochs,
            batch_size=batch_size,
            verbose=verbose,
            overwriteModel=overwriteModel,
            finishTraining=finishTraining
        )

        # Get predictions for all the data.
        pdata[key][energy_ratio_names[model_key]] = rtu.GetPredictions(regressor=regressors[model_key][key],
                                                                       model_input = All_input[key],
                                                                       indices = {k:pdata[key][k].to_numpy() for k in ['train','val','test']},
                                                                       truth = pdata[key]['cluster_E_true'].to_numpy(),
                                                                       reco = pdata[key]['clusterE'].to_numpy(),
                                                                       scaler = None,
                                                                       mapping = None,
                                                                       filename = model_filename.replace('.h5','_output.h5')
                                                                      )

## Training "simple_dnn" (depth-based) model

Here we train another simple, fully-connected neural network that uses reco energy and $\eta$ as input, along with depth information (vector of integrals of calorimeter images).

In [ ]:
# Simple model
model_key = 'simple_dnn'
skip = training_settings[model_key]['skip']

if(not skip):

    print('Starting simple (depth-based) model.')
    
    # Data preparation.
    All_input = rdu.DepthInput(pdata,
                               pcells,
                               branch_map = {
                                   's_logE':'energy',
                                   's_clusterEtaAbs':'eta'
                               }
                              )
    Split_input = rdu.DictionarySplit(All_input, pdata)
    train_input, valid_input = Split_input['train'], Split_input['val']
    # ----------------------------------------------------------------
    
    energy_ratio_names[model_key] = energy_name_prefix + model_key
    lr = training_settings[model_key]['lr']
    units = 32
    depth = 8
    decay = 0. # lr decay *within* each epoch
    dropout = 0.05 # < 0 -> no dropout
    models[model_key] = depth_network(lr=lr, decay=decay, dropout=dropout, units=units, depth=depth)

    # Set our training hyper-parameters.
    batch_size = training_settings[model_key]['batch_size']
    epochs = training_settings[model_key]['epochs']
    gamma = .1 # lr decay between epochs (via scheduler)
    patience = training_settings[model_key]['patience']
    min_delta = 0.0005
    verbose = 1
    regressors[model_key] = {}
    histories[model_key] = {}

    # Load/train the models, and evaluate them on all the data.
    for key in All_input.keys():
        print('{}: {}'.format(model_key, pi_text[key]))
        tidx = pdata[key]['train']
        vidx = pdata[key]['val']  

        model_dir = ''.join([modelpath, model_key])
        model_filename = '{}/{}{}.h5'.format(model_dir,model_key,model_filename_suffixes[key])

        regressors[model_key][key], histories[model_key][key] = rtu.TrainNetwork(
            model=models[model_key],
            modelfile = model_filename,
            x_train = train_input[key],
            y_train = pdata[key]['ratioE'][tidx],
            x_valid = valid_input[key],
            y_valid = pdata[key]['ratioE'][vidx],
            sample_weight=sample_weights[key],
            callbacks = GetCallbacks(model_filename, append=True, use_decay=True, gamma=gamma, min_delta=min_delta, patience=patience), 
            epochs=epochs,
            batch_size=batch_size,
            verbose=verbose,
            overwriteModel=overwriteModel,
            finishTraining=finishTraining
        )

        # Get predictions for all the data.
        pdata[key][energy_ratio_names[model_key]] = rtu.GetPredictions(regressor=regressors[model_key][key],
                                                                       model_input = All_input[key],
                                                                       indices = {k:pdata[key][k].to_numpy() for k in ['train','val','test']},
                                                                       truth = pdata[key]['cluster_E_true'].to_numpy(),
                                                                       reco = pdata[key]['clusterE'].to_numpy(),
                                                                       scaler = None,
                                                                       mapping = None,
                                                                       filename = model_filename.replace('.h5','_output.h5')
                                                                      )

## Simple CNN

This network uses some convolutions of the EMB layers, together with energy, eta and depth information (the depth info is computed internally).

In [ ]:
# simple CNN
model_key = 'simple_cnn'
skip = training_settings[model_key]['skip']

if(not skip):
    print('Starting simple CNN model.')
    # Data preparation.
    All_input = rdu.ResnetInput(pdata,
                                pcells,
                                branch_map = {
                                    's_logE':'energy',
                                    's_clusterEtaAbs':'eta'
                                }
    )
    Split_input = rdu.DictionarySplit(All_input, pdata)
    train_input, valid_input = Split_input['train'], Split_input['val']
    # ----------------------------------------------------------------

    energy_ratio_names[model_key] = energy_name_prefix + model_key
    lr = training_settings[model_key]['lr']
    decay = 0. # lr decay *within* each epoch
    dropout = 0.2
    augmentation = True # whether or not to augment data during training, by flipping in eta & phi
    models[model_key] = simple_cnn(lr=lr, decay=decay, dropout=dropout, augmentation=augmentation)

    # Set our training hyper-parameters.
    batch_size = training_settings[model_key]['batch_size']
    epochs = training_settings[model_key]['epochs']
    gamma = .1 # lr decay between epochs (via scheduler)
    patience = training_settings[model_key]['patience']
    min_delta = 0.0005
    verbose = 1
    regressors[model_key] = {}
    histories[model_key] = {}

    # Load/train the models, and evaluate them on all the data.
    for key in All_input.keys():
        print('{}: {}'.format(model_key, pi_text[key]))
        tidx = pdata[key]['train']
        vidx = pdata[key]['val']  

        model_dir = ''.join([modelpath, model_key])
        model_filename = '{}/{}{}.h5'.format(model_dir,model_key,model_filename_suffixes[key])

        regressors[model_key][key], histories[model_key][key] = rtu.TrainNetwork(
            model=models[model_key],
            modelfile = model_filename,
            x_train = train_input[key],
            y_train = pdata[key]['ratioE'][tidx],
            x_valid = valid_input[key],
            y_valid = pdata[key]['ratioE'][vidx],
            sample_weight=sample_weights[key],
            callbacks = GetCallbacks(model_filename, append=True, use_decay=True, gamma=gamma, min_delta=min_delta, patience=patience), 
            epochs=epochs,
            batch_size=batch_size,
            verbose=verbose,
            overwriteModel=overwriteModel,
            finishTraining=finishTraining
        )

        # Get predictions for all the data.
        pdata[key][energy_ratio_names[model_key]] = rtu.GetPredictions(regressor=regressors[model_key][key],
                                                                       model_input = All_input[key],
                                                                       indices = {k:pdata[key][k].to_numpy() for k in ['train','val','test']},
                                                                       truth = pdata[key]['cluster_E_true'].to_numpy(),
                                                                       reco = pdata[key]['clusterE'].to_numpy(),
                                                                       scaler = None,
                                                                       mapping = None,
                                                                       filename = model_filename.replace('.h5','_output.h5')
                                                                      )

## Split EMB CNN

We can train a CNN model that's inspired by one of our well-performing CNN classifiers. It groups the 6 calo layers as EMB1, EMB2+EMB3, and TileBar0+TileBar1+TileBar2. Compared to the classifier, we've removed a few convolutions to make it a little simpler, and have added in energy and abs(eta) information.

In [ ]:
# split EMB CNN (i.e. a more complicated CNN)
model_key = 'split_emb_cnn'
skip = training_settings[model_key]['skip']

if(not skip):
    print('"Starting split EMB" CNN model.')
    
    # Data preparation
    All_input = rdu.ResnetInput(pdata,
                                pcells,
                                branch_map = {
                                    's_logE':'energy',
                                    's_clusterEtaAbs':'eta'
                                }
    )
    Split_input = rdu.DictionarySplit(All_input, pdata)
    train_input, valid_input = Split_input['train'], Split_input['val']
    # ----------------------------------------------------------------
    
    energy_ratio_names[model_key] = energy_name_prefix + model_key
    lr = training_settings[model_key]['lr']
    decay = 0. # lr decay *within* each epoch
    dropout = 0.2
    augmentation = True # whether or not to augment data during training, by flipping in eta & phi
    models[model_key] = split_emb_cnn(lr=lr, decay=decay, dropout=dropout, augmentation=augmentation)

    # Set our training hyper-parameters.
    batch_size = training_settings[model_key]['batch_size']
    epochs = training_settings[model_key]['epochs']
    gamma = .1 # lr decay between epochs (via scheduler)
    patience = training_settings[model_key]['patience']
    min_delta = 0.0005
    verbose = 1
    regressors[model_key] = {}
    histories[model_key] = {}

    # Load/train the models, and evaluate them on all the data.
    for key in All_input.keys():
        print('{}: {}'.format(model_key, pi_text[key]))
        tidx = pdata[key]['train']
        vidx = pdata[key]['val']  

        model_dir = ''.join([modelpath, model_key])
        model_filename = '{}/{}{}.h5'.format(model_dir,model_key,model_filename_suffixes[key])

        regressors[model_key][key], histories[model_key][key] = rtu.TrainNetwork(
            model=models[model_key],
            modelfile = model_filename,
            x_train = train_input[key],
            y_train = pdata[key]['ratioE'][tidx],
            x_valid = valid_input[key],
            y_valid = pdata[key]['ratioE'][vidx],
            sample_weight=sample_weights[key],
            callbacks = GetCallbacks(model_filename, append=True, use_decay=True, gamma=gamma, min_delta=min_delta, patience=patience), 
            epochs=epochs,
            batch_size=batch_size,
            verbose=verbose,
            overwriteModel=overwriteModel,
            finishTraining=finishTraining
        )

        # Get predictions for all the data.
        pdata[key][energy_ratio_names[model_key]] = rtu.GetPredictions(regressor=regressors[model_key][key],
                                                                       model_input = All_input[key],
                                                                       indices = {k:pdata[key][k].to_numpy() for k in ['train','val','test']},
                                                                       truth = pdata[key]['cluster_E_true'].to_numpy(),
                                                                       reco = pdata[key]['clusterE'].to_numpy(),
                                                                       scaler = None,
                                                                       mapping = None,
                                                                       filename = model_filename.replace('.h5','_output.h5')
                                                                      )

## ResNet

We can also train an implementation of ResNet. More precisely, we use a ResNet model on the calorimeter images, and then mix in the energy and $\eta$ at the output layer.

In [ ]:
model_key = 'resnet'
skip = training_settings[model_key]['skip']

if(not skip):

    print('Starting ResNet.')
    # Data preparation -- here we can possibly take a shortcut if this data was already prepared.
    if(training_settings['split_emb_cnn']['skip']):
        All_input = rdu.ResnetInput(pdata,
                                    pcells,
                                    branch_map = {
                                        's_logE':'energy',
                                        's_clusterEtaAbs':'eta'
                                    }
        )
        Split_input = rdu.DictionarySplit(All_input, pdata)
        train_input, valid_input = Split_input['train'], Split_input['val']
    # ----------------------------------------------------------------    
    
    energy_ratio_names[model_key] = energy_name_prefix + model_key
    lr = training_settings[model_key]['lr']
    decay = 0. # lr decay *within* each epoch 1e-6
    channels = 6
    filter_sets = [
        [64,64,256],
        [128,128,512]
        #[256,256,1024],
        #[512,512,2048]
    ]         
    f_vals = [3,3] # [3,3,3,3] sizes of filters in middle of conv/identity blocks
    s_vals = [1,2] # [1,2,2,2] strides for each convolutional block
    i_vals = [2,3] # [2,3,5,2] number of identity blocks per stage
    input_shape = (128,16)
    augmentation = True # whether or not to augment data during training, by flipping in eta & phi

    models[model_key] = resnet(lr=lr, channels=channels, filter_sets=filter_sets, f_vals=f_vals, s_vals=s_vals, i_vals=i_vals, decay=decay, input_shape=input_shape, augmentation=augmentation)

    # Set our training hyper-parameters.
    batch_size = training_settings[model_key]['batch_size']
    epochs = training_settings[model_key]['epochs']
    gamma = .1 # lr decay between epochs (via scheduler)
    patience = training_settings[model_key]['patience']
    min_delta = 0.0005
    verbose = 1
    regressors[model_key] = {}
    histories[model_key] = {}

    # Load/train the models, and evaluate them on all the data.
    for key in All_input.keys():
        print('{}: {}'.format(model_key, pi_text[key]))
        tidx = pdata[key]['train']
        vidx = pdata[key]['val']  

        model_dir = ''.join([modelpath, model_key])
        model_filename = '{}/{}{}.h5'.format(model_dir,model_key,model_filename_suffixes[key])

        regressors[model_key][key], histories[model_key][key] = rtu.TrainNetwork(
            model=models[model_key],
            modelfile = model_filename,
            x_train = train_input[key],
            y_train = pdata[key]['ratioE'][tidx],
            x_valid = valid_input[key],
            y_valid = pdata[key]['ratioE'][vidx],
            sample_weight=sample_weights[key],
            callbacks = GetCallbacks(model_filename, append=True, use_decay=True, gamma=gamma, min_delta=min_delta, patience=patience), 
            epochs=epochs,
            batch_size=batch_size,
            verbose=verbose,
            overwriteModel=overwriteModel,
            finishTraining=finishTraining
        )

        # Get predictions for all the data.
        pdata[key][energy_ratio_names[model_key]] = rtu.GetPredictions(regressor=regressors[model_key][key],
                                                                       model_input = All_input[key],
                                                                       indices = {k:pdata[key][k].to_numpy() for k in ['train','val','test']},
                                                                       truth = pdata[key]['cluster_E_true'].to_numpy(),
                                                                       reco = pdata[key]['clusterE'].to_numpy(),
                                                                       scaler = None,
                                                                       mapping = None,
                                                                       filename = model_filename.replace('.h5','_output.h5')
                                                                      )

## LorentzNet

Something experimental, let's see how it does.

In [ ]:
model_key = 'lorentz'
skip = training_settings[model_key]['skip']

if(not skip):

    print('Starting LorentzNet.')
    n_vecs = [8, 8, 6, 4, 2, 2] # needs to be set for both data prep and for the network
    
    # Data preparation.
    All_input = rdu.LorentzInput(pdata,
                                 pcells,
                                 branch_map = {
                                     's_logE':'energy',
                                     's_clusterEtaAbs':'eta'
                                 },
                                 n_vecs=n_vecs
                                )
    Split_input = rdu.DictionarySplit(All_input, pdata)
    train_input, valid_input = Split_input['train'], Split_input['val']
    # ----------------------------------------------------------------    
    
    energy_ratio_names[model_key] = energy_name_prefix + model_key
    lr = training_settings[model_key]['lr']
    decay = 0. # lr decay *within* each epoch 1e-6
   
    models[model_key] = lorentz_net(lr=lr, n_vecs=n_vecs)

    # Set our training hyper-parameters.
    batch_size = training_settings[model_key]['batch_size']
    epochs = training_settings[model_key]['epochs']
    gamma = .1 # lr decay between epochs (via scheduler)
    patience = training_settings[model_key]['patience']
    min_delta = 0.0005
    verbose = 1
    regressors[model_key] = {}
    histories[model_key] = {}

    # Load/train the models, and evaluate them on all the data.
    for key in All_input.keys():
        print('{}: {}'.format(model_key, pi_text[key]))
        tidx = pdata[key]['train']
        vidx = pdata[key]['val']  

        model_dir = ''.join([modelpath, model_key])
        model_filename = '{}/{}{}.h5'.format(model_dir,model_key,model_filename_suffixes[key])

        regressors[model_key][key], histories[model_key][key] = rtu.TrainNetwork(
            model=models[model_key],
            modelfile = model_filename,
            x_train = train_input[key],
            y_train = pdata[key]['ratioE'][tidx],
            x_valid = valid_input[key],
            y_valid = pdata[key]['ratioE'][vidx],
            sample_weight=sample_weights[key],
            callbacks = GetCallbacks(model_filename, append=True, use_decay=True, gamma=gamma, min_delta=min_delta, patience=patience), 
            epochs=epochs,
            batch_size=batch_size,
            verbose=verbose,
            overwriteModel=overwriteModel,
            finishTraining=finishTraining
        )

        # Get predictions for all the data.
        pdata[key][energy_ratio_names[model_key]] = rtu.GetPredictions(regressor=regressors[model_key][key],
                                                                       model_input = All_input[key],
                                                                       indices = {k:pdata[key][k].to_numpy() for k in ['train','val','test']},
                                                                       truth = pdata[key]['cluster_E_true'].to_numpy(),
                                                                       reco = pdata[key]['clusterE'].to_numpy(),
                                                                       scaler = None,
                                                                       mapping = None,
                                                                       filename = model_filename.replace('.h5','_output.h5')
                                                                      )

## ResNet Mini

In an effort to simplify our ResNet -- and get it to train faster -- we can consider just using a single channel for our images, or some subset of channels. We can re-use the input we prepared for our full ResNet, though we'll only need a portion of it. Note that if we only use a single channel, the rescaling (via `input_shape`) is redundant, and we should just set that to the original dimensions (and, in practice, remove the scaling entirely if we stick with just one channel).

**TODO:** Reimplement this.

## Plotting results (testing how well our network works)

Now, let's plot some kinematics and network results. We'll make two groups of plots -- one for charged pions and one for neutral pions.

Within each group of plots, we'll make two plots for each quantity -- one made using just the training data, and then one made using all the data (training + whatever we excluded -- but still excluding events with `cluster_ENG_CALIB_TOT` $< 0$ since these blow up network output).

In [ ]:
# for convenience below
training_frames = {key:frame[frame['train']] for key,frame in pdata.items()}
validation_frames = {key:frame[frame['val']] for key,frame in pdata.items()}

In [ ]:
# Make plots of mse and mae for training and validation (if present)

for model_key in histories.keys():
    for dkey in histories[model_key].keys():
    
        dname = dkey
        
        epochs = len(histories[model_key][dkey]['mae'])
        epoch_ticks = epochs
        if(epoch_ticks > 10): epoch_ticks = epoch_ticks/2
        x = np.arange(epochs) + 1
        fig, ax = plt.subplots(1,2,figsize=(15,5))
    
        keys = ['mae','val_mae']
        lines = [histories[model_key][dkey][key] for key in keys]
        pu.multiplot_common(ax[0], x,lines, keys, y_min=1.0e-3, y_max=10., y_log=True, x_ticks=epoch_ticks, xlabel = 'Epoch', ylabel = 'MAE', title='Mean Avg. Error for {} ({})'.format(model_key,dname), ps=plotstyle)
    
        keys = ['mse','val_mse']
        lines = [histories[model_key][dkey][key] for key in keys]
        pu.multiplot_common(ax[1], x,lines, keys, y_min=1.0e-3, y_max=10., y_log=True, x_ticks=epoch_ticks, xlabel = 'Epoch', ylabel = 'MSE', title='Mean Sq. Error for {} ({})'.format(model_key,dname), ps=plotstyle)
    
        # add grids
        for axis in ax.flatten():
            axis.grid(True,color=plotstyle.grid_plt)

        qu.SaveSubplots(fig, ax, ['mae_{}_{}'.format(model_key,dkey), 'mse_{}_{}'.format(model_key,dkey)], savedir=plotpath)
        plt.show()